In [14]:
df['date'].min()

'2021-01-02'

In [15]:
import pandas as pd
import datetime as dt
import json
import urllib.request

all_state_names = {'ak':'Alaska','al':'Alabama','ar':'Arkansas','az':'Arizona','ca':'California','co':'Colorado',
                   'ct':'Connecticut','dc':'District of Columbia','de':'Delaware','fl':'Florida','ga':'Georgia',
                   'hi':'Hawaii','ia':'Iowa','id':'Idaho','il':'Illinois','in':'Indiana','ks':'Kansas',
                   'ky':'Kentucky','la':'Louisiana','ma':'Massachusetts','md':'Maryland','me':'Maine',
                   'mi':'Michigan','mn':'Minnesota','mo':'Missouri','ms':'Mississippi','mt':'Montana',
                   'nc':'North Carolina','nd':'North Dakota','ne':'Nebraska','nh':'New Hampshire',
                   'nj':'New Jersey','nm':'New Mexico','nv':'Nevada','ny':'New York','oh':'Ohio','ok':'Oklahoma',
                   'or':'Oregon','pa':'Pennsylvania','ri':'Rhode Island','sc':'South Carolina',
                   'sd':'South Dakota','tn':'Tennessee','tx':'Texas','ut':'Utah','va':'Virginia','vt':'Vermont',
                   'wa':'Washington','wi':'Wisconsin','wv':'West Virginia','wy':'Wyoming'}


out = []

for st in all_state_names.keys():
#     print(all_state_names[st], dt.datetime.now())
    url_state_i = 'https://covid.cdc.gov/covid-data-tracker/COVIDData/'+\
           'getAjaxData?id=integrated_county_timeseries_state_%s_external'%st.upper()
    try:
        with urllib.request.urlopen(url_state_i) as url_i:
            data_i = json.loads(url_i.read().decode())

        st_i = pd.DataFrame(data_i['integrated_county_timeseries_external_data'])
        out.append(st_i)
    except:
        print('\t','could not scrape')
        
df = pd.concat(out)
last_date = df.date.max()[:10]
df['fips'] = ("0"+df['fips_code'].astype(str).str[:]).str[-5:]
df['state'] = df['state_name']

df = df[['date','fips', 'state', 'cbsa_code', 'county',
       'new_cases_week_over_week_percent_change',
       'new_cases_7_day_rolling_average',
       'new_cases_per_100k_7_day_rolling_average',
       'new_deaths_7_day_rolling_average',
       'new_deaths_per_100k_7_day_rolling_average',
       'daily_cli_7_day_rolling_average',
       'daily_cli_percentage_7_day_rolling_average',
       'daily_ili_7_day_rolling_average',
       'daily_ili_percentage_7_day_rolling_average',
       'new_test_results_reported',
       'new_test_results_reported_7_day_rolling_average',
       'percent_new_test_results_reported_positive_7_day_rolling_average',
       'percent_positive_7_day',
       'total_test_results_reported_week_over_week_count_change',
       'testing_suppressed', 'total_hospitals_reporting',
       'admissions_covid_confirmed_last_7_days',
       'admissions_covid_confirmed_7_day_rolling_average',
       'admissions_covid_confirmed_last_7_days_per_100_beds',
       'admissions_covid_confirmed_week_over_week_percent_change',
       'percent_adult_inpatient_beds_used_confirmed_covid',
       'percent_adult_inpatient_beds_used_confirmed_covid_week_over_week_absolute_change',
       'hospitals_included_in_percent_adult_inpatient_beds_used_confirmed_covid',
       'percent_adult_icu_beds_used_confirmed_covid',
       'percent_adult_icu_beds_used_confirmed_covid_week_over_week_absolute_change',
       'hospitals_included_in_percent_adult_icu_beds_used_confirmed_covid',
       'cbsa_daily_cli_7_day_rolling_average',
       'cbsa_daily_cli_percentage_7_day_rolling_average',
       'cbsa_daily_ili_7_day_rolling_average',
       'cbsa_daily_ili_percentage_7_day_rolling_average', 
       'report_date_window_start', 'report_date_window_end',
       'community_transmission_level', 'Cases_7_day_count_change',
       'cases_per_100K_7_day_count_change', 'deaths_7_day_count_change',
       'deaths_per_100K_7_day_count_change']].sort_values(['date','state']).reset_index().iloc[:,1:]

In [32]:
df_old = pd.read_csv('../data/county_level_data_cdc.csv.gz', dtype={'fips':'str'})
df_old = df_old.loc[df_old['date']<'2021-01-02'].copy().reset_index().iloc[:,1:]

In [33]:
df = pd.concat([df_old,df[list(df_old.columns)]]).sort_values(['date','state']).reset_index().iloc[:,1:]

In [48]:
import numpy as np
for c in ['new_cases_7_day_rolling_average',
       'new_cases_per_100k_7_day_rolling_average',
       'new_deaths_7_day_rolling_average',
       'new_deaths_per_100k_7_day_rolling_average',
       'daily_cli_7_day_rolling_average',
       'daily_cli_percentage_7_day_rolling_average',
       'daily_ili_percentage_7_day_rolling_average',
       'new_test_results_reported',
       'new_test_results_reported_7_day_rolling_average',
       'percent_new_test_results_reported_positive_7_day_rolling_average',
       'percent_positive_7_day', 'total_hospitals_reporting',
       'admissions_covid_confirmed_last_7_days',
       'admissions_covid_confirmed_7_day_rolling_average',
       'admissions_covid_confirmed_last_7_days_per_100_beds',
       'admissions_covid_confirmed_week_over_week_percent_change',
       'percent_adult_inpatient_beds_used_confirmed_covid',
       'percent_adult_inpatient_beds_used_confirmed_covid_week_over_week_absolute_change',
       'hospitals_included_in_percent_adult_inpatient_beds_used_confirmed_covid',
       'percent_adult_icu_beds_used_confirmed_covid',
       'percent_adult_icu_beds_used_confirmed_covid_week_over_week_absolute_change',
       'hospitals_included_in_percent_adult_icu_beds_used_confirmed_covid']:
    df.loc[df[c]=='suppressed',c] = np.nan
    df[c] = df[c].astype(float)

# save csv: about 200+ MB of data

In [51]:
df.to_csv('../data/county_level_cdc_data_%s_2021-08-31.csv.gz'%last_date,index=False)

In [8]:
raw_testing = df[['date','fips_code','new_test_results_reported']].copy()
raw_testing['date'] = raw_testing['date'].str[:10]
raw_testing.head()

,date,fips_code,new_test_results_reported
0,2020-01-22,02013,NaN
1,2020-01-23,02013,NaN
2,2020-01-24,02013,NaN
3,2020-01-25,02013,NaN
4,2020-01-26,02013,NaN


In [9]:
raw_testing.to_csv('../data/county_level_cdc_test_volume_%s.csv'%last_date,index=False)

___________________